In [1]:
import findspark
import os
# os.environ['PYSPARK_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
# os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/hzhuang/anaconda3/bin/python'
findspark.init('/opt/cloudera/parcels/SPARK2/lib/spark2/')
import sys
import zipfile
import io
import logging
logging.getLogger().setLevel(logging.INFO)
import os
from os import path
from shutil import copyfile
from shutil import rmtree
from subprocess import call
from pyspark.sql import SparkSession



spark = SparkSession.builder.config('spark.jars', '/home/eileen/nsf_data_ingestion/libraries/spark-xml_2.11-0.5.0.jar').config("spark.executor.instances", '3').config("spark.executor.memory", '10g').config('spark.executor.cores', '3').config('spark.cores.max', '3').appName('nsf_config.tfdif').getOrCreate()

In [ ]:
import sys
sys.path.append('/home/eileen/nsf_data_ingestion/')
from nsf_data_ingestion.config import nsf_config
from nsf_data_ingestion.objects import data_source_params
import os
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/')

import pyspark
from pyspark.sql import functions as fn
from pyspark.sql.functions import regexp_extract, col, split, regexp_replace
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.ml.feature import CountVectorizer, IDF, Tokenizer, RegexTokenizer, StopWordsRemover, IDF
from pyspark.ml import Pipeline
from pyspark.sql import Row
from pyspark.sql.functions import regexp_extract, col, split, regexp_replace
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import substring
from pyspark.sql.functions import substring, length, col, expr
import requests
# getting stop words
stop_words = requests.get(nsf_config.stop_words_url).text.split()

from functools import reduce


def read_medline(spark, processed_path):
    """Creates a dataframe with the columns:
    `id`: global id
    `source`: medline
    `source_id`: PMID
    `type`: publication
    `title`
    `venue`: journal
    `abstract`
    `scientists`: authors
    `organizations`: affiliation
    `date`: publication date
    `content`: concatenation of abstract, affiliation, author, and journal
    """
    medline_path = os.path.join(processed_path, 'parquet')
    medline_df = spark.read.parquet(medline_path)
    return medline_df.select(
            fn.concat(fn.lit('medline_'), fn.col('pmid')).alias('id'),
            fn.lit('medline').alias('source'),
            fn.col('pmid').astype('string').alias('source_id'),
            fn.lit('publication').alias('type'),
            'title',
            fn.col('journal').alias('venue'),
            'abstract',
            fn.col('author').alias('scientists'),
            fn.col('affiliation').alias('organizations'),
            fn.col('pubdate').alias('date'),
            fn.concat_ws(' ',
                      fn.col('abstract'),
                      fn.col('affiliation'),
                      fn.col('author'),
                      fn.col('journal')).alias('content'),
            fn.lit(None).astype('string').alias('end_date'),
            fn.lit(None).astype('string').alias('city'),
            fn.lit(None).astype('string').alias('country'),
            fn.lit(None).astype('string').alias('other_id')
    )

def add_rowid(x):
    """Called on a RDD when zipWithIndex() is used"""
    return Row(row_id = x[1], **x[0].asDict())

def fit_tfidf_pipeline(content_df):
    tokenizer = RegexTokenizer(). \
        setGaps(False). \
        setPattern('\\p{L}+'). \
        setInputCol('content'). \
        setOutputCol('words')

    sw = StopWordsRemover() \
        .setStopWords(stop_words) \
        .setCaseSensitive(False) \
        .setInputCol("words") \
        .setOutputCol("filtered")

    cv = CountVectorizer(). \
        setInputCol('filtered'). \
        setOutputCol('tf'). \
        setMinTF(1). \
        setMinDF(10). \
        setVocabSize(2 ** 17)

    # fit dataframe_df
    cv_transformer = Pipeline(stages=[tokenizer, sw, cv]).fit(content_df)

    idf = IDF(minDocFreq=10). \
        setInputCol('tf'). \
        setOutputCol('tfidf')

    tfidf_transformer = Pipeline(stages=[cv_transformer, idf]).fit(content_df)

    return tfidf_transformer

def main(data_source):
    processed_path = '/user/eileen/medline/'
    medline_df = read_medline(spark, processed_path)

    spark.stop()
    

In [1]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2/lib/spark2/')
import zipfile
import zipimport
import io
import logging
logging.getLogger().setLevel(logging.INFO)
from pyspark.sql import functions
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import Window
from pyspark.sql.functions import rank, max, sum, desc
import zlib
import importlib
import os
from os import path
from shutil import copyfile
from shutil import rmtree
from subprocess import call
import sys
# sys.path.append('/home/ananth/nsf_data_ingestion/')
sys.path.append('/home/eileen/nsf_data_ingestion/')
from nsf_data_ingestion.config import spark_config
from nsf_data_ingestion.objects import data_source_params


# def create_session(libraries_list):
#     logging.info('Creating Spark Session....')
#     spark = SparkSession.builder.config("spark.executor.instances", spark_config.exec_instance).\
#                                  config("spark.executor.memory", spark_config.exec_mem).\
#                                  config('spark.executor.cores', spark_config.exec_cores).\
#                                  config('spark.cores.max', spark_config.exec_max_cores).\
#                                  appName(data_source_name).getOrCreate()
#     spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/pubmed_parser-0.1.0-py3.6.egg')
#     spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/Unidecode-1.1.1-py3.6.egg')
# #     for library in libraries_list:
# #         logging.info('Adding Libraries' + str(library))
# #         spark.sparkContext.addPyFile(library)    # adding libraries
# #         spark.sparkContext.addPyFile(library)

        
#     return spark

def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    print(pp.__file__)
    print(unidecode.__file__)
    
    filepath = gzip_str[0]
    gzip_content = gzip_str[1]
    _, file_name = path.split(filepath)
    # decompress gzip

    xml_string = gzip_content.split("\n", 3)[3];
    articles = pp.parse_medline_xml(xml_string)
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]


if __name__ == '__main__':
    data_source_name = 'medline'
    params_list = data_source_params.mapping.get(data_source_name)
    medline_xml_path= params_list.get('xml_path')
    print(medline_xml_path)
    medline_parquet_path= params_list.get('parquet_path')
    libraries_list = spark_config.libraries_list
    
    print("Reading from {} and writing to {}.".format(medline_xml_path, medline_parquet_path))
    
    spark = create_session(libraries_list)
    
    hdfs_data = spark.sparkContext.wholeTextFiles(os.path.join(medline_xml_path, '*.xml.gz'), minPartitions=10000)
    preprocess = hdfs_data.flatMap(parse_gzip_medline_str)
    medline_df = preprocess.toDF()
    
    window = Window.partitionBy(['pmid']).orderBy(desc('file_name'))
    ##only get the last version of documents
    last_medline_df = medline_df.select(
        max('delete').over(window).alias('is_deleted'),
        rank().over(window).alias('pos'), '*').\
        where('is_deleted = False and pos = 1').\
        drop('is_deleted').drop('pos').drop('delete')
    
    if not call(["hdfs", "dfs", "-test", "-d", medline_parquet_path]):
            logging.info('Parquet Files Exist Deleting .......')
            call(["hdfs", "dfs", "-rm", "-r", "-f", medline_parquet_path])
            
    logging.info('Writing New parquet Files .......')
    medline_df.write.parquet(medline_parquet_path)
    medline_df.head(10)
    
    spark.stop()

INFO:root:Creating Spark Session....


/user/eileen/medline/medline_data/
Reading from /user/eileen/medline/medline_data/ and writing to /user/eileen/medline/parquet/.


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3, ist-deacuna-n5.syr.edu, executor 2): java.io.IOException: Cannot run program "/home/hzhuang/anaconda3/bin/python": error=2, No such file or directory
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:197)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:122)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:95)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:108)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.R

In [11]:
import findspark
import os
os.environ['PYSPARK_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/hzhuang/anaconda3/bin/python'
findspark.init('/opt/cloudera/parcels/SPARK2/lib/spark2/')
import sys
import zipfile
import io
import logging
logging.getLogger().setLevel(logging.INFO)
import os
from os import path
from shutil import copyfile
from shutil import rmtree
from subprocess import call
from pyspark.sql import SparkSession
#os.environ['PYSPARK_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/hzhuang/anaconda3/bin/python'


spark = SparkSession.builder.config('spark.jars', '/home/eileen/nsf_data_ingestion/libraries/spark-xml_2.11-0.5.0.jar').config("spark.executor.instances", '3').config("spark.executor.memory", '10g').config('spark.executor.cores', '3').config('spark.cores.max', '3').appName('nsf_config.tfdif').getOrCreate()

spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/pubmed_parser-0.1.0-py3.6.egg')
spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/Unidecode-1.1.1-py3.6.egg')

sc = spark.sparkContext

In [12]:
def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    print(pp.__file__)
    print(unidecode.__file__)
    
    filepath = gzip_str[0]
    gzip_content = gzip_str[1]
    _, file_name = path.split(filepath)
    # decompress gzip

    xml_string = gzip_content.split("\n", 3)[3];
    articles = pp.parse_medline_xml(xml_string)
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]


if __name__ == '__main__':
    data_source_name = 'medline'
    params_list = data_source_params.mapping.get(data_source_name)
    medline_xml_path= params_list.get('xml_path')
    print(medline_xml_path)
    medline_parquet_path= params_list.get('parquet_path')
    libraries_list = spark_config.libraries_list
    
    print("Reading from {} and writing to {}.".format(medline_xml_path, medline_parquet_path))
    
    #spark = create_session(libraries_list)
    
    hdfs_data = spark.sparkContext.wholeTextFiles(os.path.join(medline_xml_path, '*.xml.gz'), minPartitions=10000)
    preprocess = hdfs_data.flatMap(parse_gzip_medline_str)
    medline_df = preprocess.toDF()
    
    window = Window.partitionBy(['pmid']).orderBy(desc('file_name'))
    ##only get the last version of documents
    last_medline_df = medline_df.select(
        max('delete').over(window).alias('is_deleted'),
        rank().over(window).alias('pos'), '*').\
        where('is_deleted = False and pos = 1').\
        drop('is_deleted').drop('pos').drop('delete')
    
    if not call(["hdfs", "dfs", "-test", "-d", medline_parquet_path]):
            logging.info('Parquet Files Exist Deleting .......')
            call(["hdfs", "dfs", "-rm", "-r", "-f", medline_parquet_path])
            
    logging.info('Writing New parquet Files .......')
    medline_df.write.parquet(medline_parquet_path)
    medline_df.head(10)
    
    spark.stop()

/user/eileen/medline/medline_data/
Reading from /user/eileen/medline/medline_data/ and writing to /user/eileen/medline/parquet/.


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 4 times, most recent failure: Lost task 0.3 in stage 6.0 (TID 27, ist-deacuna-n8.syr.edu, executor 6): java.io.IOException: Cannot run program "/home/hzhuang/anaconda3/bin/python": error=2, No such file or directory
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:197)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:122)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:95)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:108)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.

In [5]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2/lib/spark2/')
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator
import os
import sys
sys.path.append('/home/eileen/nsf_data_ingestion/')
from nsf_data_ingestion.pubmed import pubmed_download
from nsf_data_ingestion.pubmed.pubmed_download import *
from nsf_data_ingestion.federal_reporter import federal_reporter_download
from nsf_data_ingestion.grants_gov import grants_gov_download_put_hdfs
from nsf_data_ingestion.grants_gov.grants_gov_download_put_hdfs import *
from nsf_data_ingestion.federal_reporter.federal_reporter_download import *
from nsf_data_ingestion.medline import medline_download
from nsf_data_ingestion.medline.medline_download import *
from nsf_data_ingestion.medline import medline_parquet_write
from nsf_data_ingestion.federal_reporter import federal_parquet_write
from nsf_data_ingestion.grants_gov import write_to_parquet_grants
from nsf_data_ingestion.models import tfidf_aggregate
from nsf_data_ingestion.models import large_scale_svd
from nsf_data_ingestion.arxiv import arxiv_download
from nsf_data_ingestion.arxiv import arxiv_parquet_write
from nsf_data_ingestion.kimun_loader import kimun_loader
from nsf_data_ingestion.arxiv.arxiv_download import *

from nsf_data_ingestion.objects import data_source_params
from nsf_data_ingestion.utils import utils_functions

from nsf_data_ingestion.config.spark_config import *

#from nsf_data_ingestion.models import tfidf_aggregate
from datetime import datetime, timedelta
import subprocess
from subprocess import call

import sys
sys.path.append('/home/eileen/nsf_data_ingestion/')
from nsf_data_ingestion.config import nsf_config
from nsf_data_ingestion.objects import data_source_params
import os
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2-2.3.0.cloudera3-1.cdh5.13.3.p0.458809/lib/spark2/')

import pyspark
from pyspark.sql import functions as fn
from pyspark.sql.functions import regexp_extract, col, split, regexp_replace
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.ml.feature import CountVectorizer, IDF, Tokenizer, RegexTokenizer, StopWordsRemover, IDF
from pyspark.ml import Pipeline
from pyspark.sql import Row
from pyspark.sql.functions import regexp_extract, col, split, regexp_replace
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import substring
from pyspark.sql.functions import substring, length, col, expr
import requests
# getting stop words
stop_words = requests.get(nsf_config.stop_words_url).text.split()

from functools import reduce

# def create_spark_session(name):
#     spark = SparkSession.builder.config("spark.executor.instances", '3')\
#     .config("spark.executor.memory", '30g')\
#     .config('spark.executor.cores', '7')\
#     .config('spark.cores.max', '7')\
#     .appName('tfdf')\
#     .getOrCreate()
#     return spark
# def create_spark_session(name):
#     spark = SparkSession.builder.config("spark.executor.instances", '2')\
#     .config("spark.shuffle.service.enabled", "false")\
#     .config("spark.dynamicAllocation.enabled", "false")\
#     .config("spark.cores.max", "1")\
#     .config("spark.executor.cores","1")\
#     .appName('tfdf')\
#     .getOrCreate()
#     return spark

def create_spark_session(name):
    spark = SparkSession.builder.config("spark.executor.instances", '3')\
    .config("spark.shuffle.service.enabled", "false")\
    .config("spark.executor.memory", '30g')\
    .config("spark.dynamicAllocation.enabled", "false")\
    .config("spark.cores.max", "7")\
    .config("spark.executor.cores","7")\
    .appName('tfdif')\
    .getOrCreate()
    return spark
def read_medline(spark, processed_path):
    """Creates a dataframe with the columns:
    `id`: global id
    `source`: medline
    `source_id`: PMID
    `type`: publication
    `title`
    `venue`: journal
    `abstract`
    `scientists`: authors
    `organizations`: affiliation
    `date`: publication date
    `content`: concatenation of abstract, affiliation, author, and journal
    """
    medline_path = os.path.join(processed_path, 'parquet')
    medline_df = spark.read.parquet(medline_path)
    return medline_df.select(
            fn.concat(fn.lit('medline_'), fn.col('pmid')).alias('id'),
            fn.lit('medline').alias('source'),
            fn.col('pmid').astype('string').alias('source_id'),
            fn.lit('publication').alias('type'),
            'title',
            fn.col('journal').alias('venue'),
            'abstract',
            fn.col('author').alias('scientists'),
            fn.col('affiliation').alias('organizations'),
            fn.col('pubdate').alias('date'),
            fn.concat_ws(' ',
                      fn.col('abstract'),
                      fn.col('affiliation'),
                      fn.col('author'),
                      fn.col('journal')).alias('content'),
            fn.lit(None).astype('string').alias('end_date'),
            fn.lit(None).astype('string').alias('city'),
            fn.lit(None).astype('string').alias('country'),
            fn.lit(None).astype('string').alias('other_id')
    )
# # def read_medline(spark, processed_path):
# #     """Creates a dataframe with the columns:
# #     `id`: global id
# #     `source`: medline
# #     `source_id`: PMID
# #     `type`: publication
# #     `title`
# #     `venue`: journal
# #     `abstract`
# #     `scientists`: authors
# #     `organizations`: affiliation
# #     `date`: publication date
# #     `content`: concatenation of abstract, affiliation, author, and journal
# #     """
# #     medline_path = processed_path
# #     medline_df = spark.read.parquet(medline_path)
# #     return medline_df.select(
# #             fn.concat(fn.lit('medline_'), fn.col('pmid')).alias('id'),
# #             fn.lit('medline').alias('source'),
# #             fn.col('pmid').astype('string').alias('source_id'),
# #             fn.lit('publication').alias('type'),
# #             'title',
# #             fn.col('journal').alias('venue'),
# #             'abstract',
# #             fn.col('author').alias('scientists'),
# #             fn.col('affiliation').alias('organizations'),
# #             fn.col('pubdate').alias('date'),
# #             fn.concat_ws(' ',
# #                       fn.col('abstract'),
# #                       fn.col('affiliation'),
# #                       fn.col('author'),
# #                       fn.col('journal')).alias('content'),
# #             fn.lit(None).astype('string').alias('end_date'),
# #             fn.lit(None).astype('string').alias('city'),
# #             fn.lit(None).astype('string').alias('country'),
# #             fn.lit(None).astype('string').alias('other_id')
# #     )

# def read_federal_exporter(spark, fed_processed_path):
#     """Creates a dataframe with the columns:
#     `id`: global id
#     `source`: federal_exporter
#     `source_id`: project id
#     `type`: grant
#     `title`: PROJECT_TITLE
#     `venue`: AGENCY
#     `abstract`
#     `scientists`: CONTACT_PI + OTHER_PIS
#     `organizations`: ORGANIZATION_NAME
#     `date`: BUDGET_START_DATE
#     `content`: concatenation of abstract, title, PIs, agency, and organization name
#     `end_date`: BUDGET_END_DATE
#     `city`: ORGANIZATION_CITY
#     `country`: ORGANIZATION_COUNTRY
#     `other_id`: PROJECT_NUMBER
#     """
#     abstracts_df = spark.read.parquet(os.path.join(fed_processed_path, 'abstracts.parquet'))
#     projects_df = spark.read.parquet(os.path.join(fed_processed_path, 'projects.parquet'))
#     together_df = projects_df.join(abstracts_df, 'PROJECT_ID')
#     return together_df.select(fn.concat(fn.lit('fe_'), fn.col('PROJECT_ID')).alias('id'),
#         fn.lit('federal_exporter').alias('source'),
#         fn.col('PROJECT_ID').astype('string').alias('source_id'),
#         fn.lit('grant').alias('type'),
#         fn.col('PROJECT_TITLE').alias('title'),
#         fn.col('AGENCY').alias('venue'),
#         fn.col('ABSTRACT').alias('abstract'),
#         fn.concat_ws('; ',
#                      fn.col('CONTACT_PI_PROJECT_LEADER'),
#                      fn.col('OTHER_PIS')).alias('scientists'),
#         fn.col('ORGANIZATION_NAME').astype('string').alias('organizations'),
#         fn.col('BUDGET_START_DATE').alias('date'),
#         fn.concat_ws(' ',
#                   fn.col('ABSTRACT'),
#                   fn.col('PROJECT_TITLE'),
#                   fn.concat_ws(' ', fn.col('CONTACT_PI_PROJECT_LEADER'), fn.col('OTHER_PIS')),
#                   fn.col('AGENCY'),
#                   fn.col('ORGANIZATION_NAME')
#             ).alias('content'),
#         fn.col('BUDGET_END_DATE').alias('end_date'),
#         fn.col('ORGANIZATION_CITY').alias('city'),
#         fn.col('ORGANIZATION_COUNTRY').alias('country'),
#         fn.col('PROJECT_NUMBER').alias('other_id')
# )

# # def read_arxiv(spark, processed_path):
# #     """Creates a dataframe with the columns:
# #     `id`: global id
# #     `source`: arxiv
# #     `source_id`: arxiv id
# #     `type`: publication
# #     `title`
# #     `venue`: concatenation of subjects
# #     `abstract`
# #     `scientists`: authors
# #     `organizations`: null
# #     `date`: publication date
# #     `content`: concatenation of abstract, affiliation, author, and journal
# #     """
# #     arxiv_path = os.path.join(processed_path)
# #     arxiv_df = spark.read.parquet(arxiv_path)
# #     return arxiv_df.select(
# #         fn.concat(fn.lit('arxiv_'), fn.col('id')).alias('id'),
# #         fn.lit('arxiv').alias('source'),
# #         fn.col('id').astype('string').alias('source_id'),
# #         fn.lit('publication').alias('type'),
# #         'title',
# #         fn.concat_ws('; ', 'subjects').alias('venue'),
# #         'abstract',
# #         fn.concat_ws(';', 'authors').alias('scientists'),
# #         fn.lit(None).astype('string').alias('organizations'),
# #         fn.col('datastamp').alias('date'),
# #         fn.concat_ws(' ',
# #                   fn.col('abstract'),
# #                   fn.col('title'),
# #                   fn.concat_ws(' ', 'authors'),
# #                   fn.concat_ws(' ', 'subjects')).alias('content'),
# #         fn.lit(None).astype('string').alias('end_date'),
# #         fn.lit(None).astype('string').alias('city'),
# #         fn.lit(None).astype('string').alias('country'),
# #         fn.lit(None).astype('string').alias('other_id')
# #     )
# def read_arxiv(spark, arxiv_path):
#     """Creates a dataframe with the columns:
#     `id`: global id
#     `source`: arxiv
#     `source_id`: arxiv id
#     `type`: publication
#     `title`
#     `venue`: concatenation of subjects
#     `abstract`
#     `scientists`: authors
#     `organizations`: null
#     `date`: publication date
#     `content`: concatenation of abstract, affiliation, author, and journal
#     """
#     arxiv_path = os.path.join(arxiv_path, 'parquet')
#     arxiv_df = spark.read.parquet(arxiv_path)
#     return arxiv_df.select(
#         fn.concat(fn.lit('arxiv_'), fn.col('id')).alias('id'),
#         fn.lit('arxiv').alias('source'),
#         fn.col('id').astype('string').alias('source_id'),
#         fn.lit('publication').alias('type'),
#         'title',
#         fn.concat_ws('; ', 'subjects').alias('venue'),
#         'abstract',
#         fn.concat_ws(';', 'authors').alias('scientists'),
#         fn.lit(None).astype('string').alias('organizations'),
#         fn.col('datastamp').alias('date'),
#         fn.concat_ws(' ',
#                   fn.col('abstract'),
#                   fn.col('title'),
#                   fn.concat_ws(' ', 'authors'),
#                   fn.concat_ws(' ', 'subjects')).alias('content'),
#         fn.lit(None).astype('string').alias('end_date'),
#         fn.lit(None).astype('string').alias('city'),
#         fn.lit(None).astype('string').alias('country'),
#         fn.lit(None).astype('string').alias('other_id')
#     )


# def read_grants_gov(spark, processed_path_grants):
#     """Creates a dataframe with the columns:
#     `id`: global id
#     `source`: grants_gov
#     `source_id`: project id
#     `type`: grant
#     `title`: PROJECT_TITLE
#     `venue`: AGENCY
#     `abstract`
#     `scientists`: CONTACT_PI + OTHER_PIS
#     `organizations`: ORGANIZATION_NAME
#     `date`: BUDGET_START_DATE
#     `content`: concatenation of abstract, title, PIs, agency, and organization name
#     `end_date`: BUDGET_END_DATE
#     `city`: ORGANIZATION_CITY
#     `country`: ORGANIZATION_COUNTRY
#     `other_id`: PROJECT_NUMBER
#     """
#     print("processed path : ", processed_path_grants)
#     synopsis_df = spark.read.parquet(os.path.join(processed_path_grants, 'synopsis.parquet'))
#     synopsis_df=synopsis_df.withColumn("GrantorContactName",fn.col("GrantorContactText"))
#     forecast_df = spark.read.parquet(os.path.join(processed_path_grants, 'forecast.parquet'))
#     forecast_df=forecast_df.drop("OpportunityTitle","AgencyCode","AgencyName","AdditionalInformationOnEligibility","OpportunityNumber","Description","PostDate", "CloseDate",'GrantorContactName')
#     together_df=synopsis_df.join(forecast_df, "OpportunityID", "outer")
    
#     return together_df.select(fn.concat(fn.lit('gg_'), fn.col('OpportunityID')).alias('id'),
#             fn.lit('grants_gov').alias('source'),
#             fn.col('OpportunityID').astype('string').alias('source_id'),
#             fn.lit('grant').alias('type'),
#             fn.col('OpportunityTitle').alias('title'),
#             fn.col('AgencyCode').alias('venue'),
#             fn.col('AdditionalInformationOnEligibility').alias('abstract'),
#             fn.col('GrantorContactName').alias('scientists'),
#             fn.col('AgencyName').astype('string').alias('organizations'),
#             fn.col('PostDate').alias('date'),#PostDate
# #             fn.concat_ws(' ',
# #                       fn.col('AdditionalInformationOnEligibility'),
# #                       fn.col('OpportunityTitle'),
# #                       fn.col('GrantorContactName'),
# #                       fn.col('AgencyCode'),
# #                       fn.col('AgencyName')
# #                 ).alias('content'),
#             fn.col('Description').alias('content'),
#             fn.col('CloseDate').alias('end_date'),#CloseDate
#             fn.lit('NewYork').alias('city'),
#             fn.lit('United States').alias('country'),
#             fn.col('OpportunityNumber').alias('other_id')
#     )



def add_rowid(x):
    """Called on a RDD when zipWithIndex() is used"""
    return Row(row_id = x[1], **x[0].asDict())

def fit_tfidf_pipeline(content_df):
    tokenizer = RegexTokenizer(). \
        setGaps(False). \
        setPattern('\\p{L}+'). \
        setInputCol('content'). \
        setOutputCol('words')

    sw = StopWordsRemover() \
        .setStopWords(stop_words) \
        .setCaseSensitive(False) \
        .setInputCol("words") \
        .setOutputCol("filtered")

    cv = CountVectorizer(). \
        setInputCol('filtered'). \
        setOutputCol('tf'). \
        setMinTF(1). \
        setMinDF(10). \
        setVocabSize(2 ** 17)

    # fit dataframe_df
    cv_transformer = Pipeline(stages=[tokenizer, sw, cv]).fit(content_df)

    idf = IDF(minDocFreq=10). \
        setInputCol('tf'). \
        setOutputCol('tfidf')

    tfidf_transformer = Pipeline(stages=[cv_transformer, idf]).fit(content_df)

    return tfidf_transformer

def main(data_source):
    processed_path = '/user/eileen/medline/'
    
    fed_processed_path = '/user/eileen/federal/xml/'
    #tfidf_path = '/user/ananth/tdifupdate/'
    arxiv_path = '/user/eileen/arxiv/'
   

    

    ###################sourabh ghosh code#######################
    models_path = '/user/eileen/'
    tfidf_path = '/user/eileen/tfidf.parquet'
    
    #location where processed synopsis parquet for grants.gov are placed
    processed_path_grants = '/user/eileen/grants/'    

    
    spark = create_spark_session('tfidf-computation grants')

    #grantsgovs_df = read_grants_gov(spark,os.path.join(processed_path_grants))
#     arxiv_df = read_arxiv(spark, arxiv_path)
#     arxiv_df.head(10)
    medline_df = read_medline(spark, processed_path)
    fe_df = read_federal_exporter(spark, fed_processed_path)
#     grantsgovs_df = grantsgovs_df.filter((grantsgovs_df["content"].isNotNull())).\
#                                 filter("content != ' '").\
#                                     filter("content != ''")
#     split_col = pyspark.sql.functions.split(grantsgovs_df['scientists'], '&')

#     grantsgovs_df = grantsgovs_df.withColumn('scientists', split_col.getItem(0))
#     split_col1 = pyspark.sql.functions.split(grantsgovs_df['scientists'], '[0-9]')
#     grantsgovs_df = grantsgovs_df.withColumn('scientists', split_col1.getItem(0))
#     grantsgovs_df = grantsgovs_df.withColumn("date", grantsgovs_df["date"].cast("string"))
#     grantsgovs_df = grantsgovs_df.withColumn("date", grantsgovs_df["end_date"].cast("string"))
#     grantsgovs_df = grantsgovs_df.withColumn("date",expr("substring(date, 4, length(date)-3)"))
#     grantsgovs_df = grantsgovs_df.withColumn("end_date",expr("substring(end_date, 4, length(end_date)-3)"))
    
#     dataframe_list = [medline_df, fe_df,arxiv_df]
#     #dataframe_list = [medline_df, grantsgovs_df, grantsgovs_df]
#     all_data_df = content_df = reduce(DataFrame.unionAll, dataframe_list)

#     tfidf_transformer = fit_tfidf_pipeline(all_data_df)
#     tfidf_model_path = os.path.join(models_path, 'tfidf_transformer.model')

#     tfidf_transformer.write().overwrite().save(tfidf_model_path)
#     tfidf_df = tfidf_transformer.transform(all_data_df). \
#             select(all_data_df.columns + ['tfidf'])
#     tfidf_df.write.parquet(tfidf_path, mode='overwrite')
    spark.stop()
    

In [8]:
main(nsf_config.tfdif)

Py4JError: An error occurred while calling None.org.apache.spark.api.python.PythonAccumulatorV2. Trace:
py4j.Py4JException: Constructor org.apache.spark.api.python.PythonAccumulatorV2([class java.lang.String, class java.lang.Integer]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)

